In [40]:
import pandas as pd
from fuzzywuzzy import fuzz
import itertools
import random

path = '/Users/zfrancis/Documents/Personal_Work_Stuff/programs/odsi/DOGL_Repo/DOGL/cleaning-tools/Table-cleaning/data'

# Read the CSV
partners_list_df = pd.read_csv(path+'/partner_data_raw.csv')

# Duplicate the first column
partners_list_df['cleaned'] = partners_list_df.iloc[:, 0]

# Functions for quickly deleting and replacing one- or few-time errors
def quick_remove(text):
    partners_list_df['cleaned'] = partners_list_df['cleaned'].str.replace(text,'')
    return partners_list_df

def quick_replace(old_text, new_text):
    partners_list_df['cleaned'] = partners_list_df['cleaned'].str.replace(old_text, new_text)
    return partners_list_df

# Solve all those little errors
partners_list_df = quick_remove('"')
partners_list_df = quick_remove("Partnerships:' ")
partners_list_df = quick_replace(" 'Cooperations:'",";")
partners_list_df = quick_remove("Advisory Board: ")
partners_list_df = quick_replace("Does not explictly state the name of these partners", "Not indicated")
partners_list_df = quick_replace("Includes a weblink to 'list of partners' that lists categories of Memberstates, Scientific Institutions, Civil Society, Business, Foundations. Only one partner is listed, Tara Ocean Foundation. https://ioc.unesco.org/partners IODE is the data collection/management entity/programme of IOC. See IODE for list of Partners.","Tara Ocean Foundation")
partners_list_df = quick_remove("OCEAN DECADE PARTNERS: ")

def comma_to_semicolon(row):
    original_text = partners_list_df.loc[row, 'cleaned']
    new_text = original_text.replace(',',';')
    partners_list_df.loc[row, 'cleaned'] = new_text


# On certain rows, replace commas with semicolons
comma_rows = [31,34,40,41,42,43,44,45,46,51,57]
for row in comma_rows:
    comma_to_semicolon(row)

# Replace NaN values with "Not Indicated"
partners_list_df['cleaned'].fillna("Not Indicated", inplace=True)

# Display the cleaned data to check the results
partners_list_df[['cleaned']].head()


canonical_df = pd.read_csv(path+'/canonical_partners.csv',encoding='latin9')
check_names = pd.DataFrame(columns=['Entry Name','Canonical Name','Score'])

def find_canonical_name(org_name, threshold=95):
    max_score = 0
    canonical_name_to_return = org_name  # Default to original name
    
    for canonical_name in canonical_df['canonical_name']:
        score = fuzz.token_set_ratio(org_name, canonical_name)
        
        if score > max_score:
            max_score = score
            canonical_name_to_return = canonical_name
        
        # Overwrite the name if similarity exceeds threshold
        if max_score > threshold:
            return canonical_name_to_return
        
        elif score > threshold - 5: # If the score is > 90, put it in the manual review dataframe
            check_names.loc[len(check_names.index)] = [org_name, canonical_name, score]
    
    return org_name

def process_canonical(org_names_str): # Processes the canonical naming convention on each row
    org_names = org_names_str.split(';')
    cleaned_names = [find_canonical_name(name.strip()) for name in org_names]
    return '; '.join(cleaned_names)

# Apply the function to your data
partners_list_df['cleaned'] = partners_list_df['cleaned'].apply(process_canonical)


#names_to_check = pd.DataFrame.from_records(check_names)
#names_to_check.to_csv(path+'/names_to_check.csv')
checked_names = pd.read_csv(path+'/checked_names.csv')

for _, row in check_names.iterrows():
    if row['Score'] == 1:
        partners_list_df = quick_replace(row['Entry Name'], row['Canonical Name'])
    

# Replace some extraneous ones that didn't get solved on the first pass
partners_list_df = quick_replace('United Nations Environment Program (UNEP)','United Nations Environment Programme (UNEP)')
partners_list_df = quick_replace('National Oceanic and Atmospheric Administration, United States of America', 'National Oceanic & Atmospheric Administration (NOAA)')
partners_list_df = quick_replace('National Aeronautic and Space Administration (NASA) Ames','National Aeronautics and Space Administration (NASA)')
partners_list_df = quick_replace('US National Aeronautics and Space Administration (NASA)', 'National Aeronautics and Space Administration (NASA)')
partners_list_df = quick_replace('International Council for Science (ICSU)', 'International Science Council (ISC)') # These two merged as the ISC
partners_list_df = quick_replace('National Oceanographic & Atmospheric Administration (NOAA)','National Oceanographic and Atmospheric Administration (NOAA)')
partners_list_df = quick_replace('None indicated', 'Not indicated')
partners_list_df = quick_replace('World Wildlife Foundation (WWF)','World Wildlife Fund (WWF)')
partners_list_df = quick_replace('International Oceanographic Commission (IOC)','Intergovernmental Oceanographic Commission (IOC)')

# Test code by getting current matches after all cleaning
def get_matches(df, col):
    individual_orgs = df[col].str.split(';').explode().str.strip()
    unique_orgs = individual_orgs.unique()
    potential_matches = []

    for org1, org2 in itertools.combinations(unique_orgs, 2): # Iterates through all unique pairs of org names
        score = fuzz.token_set_ratio(org1, org2)
        
        # If similarity score is above 90 but not 100, store a potential match
        if score > 90 and score < 100:
            potential_matches.append((org1, org2, score))

    # Creating a DataFrame for better visualization of potential matches
    matches_df = pd.DataFrame(potential_matches, columns=['Org1', 'Org2', 'Score']).sort_values(by='Score', ascending=False)
    return matches_df

matches_df = get_matches(partners_list_df, 'cleaned')

matches_df.to_csv(path+'/partner_matches.csv')


partners_list_df.to_csv(path+'/partner_data_cleaned.csv')

/var/folders/2p/0hr75ykx133b0bk5k4nf52580000gn/T/ipykernel_62939/352259237.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  partners_list_df['cleaned'] = partners_list_df['cleaned'].str.replace(old_text, new_text)
/var/folders/2p/0hr75ykx133b0bk5k4nf52580000gn/T/ipykernel_62939/352259237.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  partners_list_df['cleaned'] = partners_list_df['cleaned'].str.replace(old_text, new_text)


23    Comoros; Kenya; Madagascar; Mauritius; Mozambi...
80                                  National Geographic
90    National Oceanic and Atmospheric Administratio...
88    International Association of Aquatic and Marin...
17    Centre for Documentation, Research and Experim...
32    National Oceanic and Atmospheric Administratio...
38                                        Not Indicated
13    Jeff Ardron, Commonwealth Secretariat, UK; Jon...
53    Accenture; Aker ASA; Aker Biomarine; AkerBP; A...
87    Indian National Center for Ocean Information S...
Name: cleaned, dtype: object
